In [2]:
!pip install tensorflow

In [3]:
!pip install tensorflow[and-cuda]

In [3]:
# IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.13.1'

# Part 1 - Data Preprocessing

In [5]:
# IMPORTING DATASET
dataset = pd.read_csv(r'C:\Users\anand\OneDrive - Sheridan College\College\Machine Learning\Deep Learning\Artificial Neural Network\Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [6]:
print(x)
print("-------------------------------------------")
print("-------------------------------------------")
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
-------------------------------------------
-------------------------------------------
[1 0 1 ... 1 1 0]


In [7]:
# ENCODING CATEGORICAL DATA

# Label Encoding the "Gender" Column

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])  # All the rows of only the second column
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [8]:
# One Hot Encoding the "Geography Column"

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder()
ct = ColumnTransformer(transformers=[('encoder', onehotencoder, [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [9]:
# SPLITTING THE DATASET into TRAINING AND TESTING SET
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [10]:
# FEATURE SCALING

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Part 2 - Building the ANN

In [11]:
# INITIALIZING THE ANN

ann = tf.keras.models.Sequential()

In [12]:
# ADDING THE INPUT LAYER AND THE FIRST HIDDEN LAYER

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [13]:
# ADDING THE SECOND HIDDEN LAYER

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [14]:
# ADDING THE OUTPUT LAYER

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) # For non-binary it's - softmax

# Part 3 - Training the ANN

In [15]:
# Compiling the ANN - with a loss function an optimizer

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # For non-binary it's - categorical_crossentropy

# Adam (Adaptive Moment Estimation) is a stochastic gradient descent used to update network weights iteratively based on training data

In [16]:
# Training the ANN on the Trainer Set

ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.7345 - accuracy: 0.5150
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5431 - accuracy: 0.7971
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4567 - accuracy: 0.8100
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4252 - accuracy: 0.8170
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4108 - accuracy: 0.8215
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3967 - accuracy: 0.8320
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3834 - accuracy: 0.8397
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3734 - accuracy: 0.8439
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3664 - accuracy: 0.8459
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3621 - accura

In [25]:
# Using ANN model to predict whether a specific customer will leave the bank or not
print(f'The probability is:  {ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))}')
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 17ms/step
The probability is:  [[0.03260542]]
1/1 [==============================] - 0s 16ms/step
[[False]]


# Part 4 - Making the Predictions and Evaluating the Model

In [26]:
# Predicting the Test Set Results

y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 955us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [27]:
# Making the Confusion Matrix

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1524   71]
 [ 207  198]]


0.861